## <font color="white">Etude: <font color="red"> Faire un résumé statistique descriptive du pourcentage des jobs les plus demandés, des jobs moyennement demandés, les jobs les moins demandées et le top 5 des pays où ces jobs sont le plus demandés.

### Requêtage de la donnée sur RDS postgreSQL

In [1]:
import os
from dotenv import load_dotenv
import pandas as pd
from sqlalchemy import create_engine

# Charger le fichier .env
load_dotenv(dotenv_path="C:/Users/Эли Жоэль/Downloads/PROJET CLOUD/.env")

# Récupérer les bonnes variables (adaptées à ton .env)
user = os.getenv("PG_USER")
password = os.getenv("PG_PASSWORD")
host = os.getenv("PG_HOST")
port = os.getenv("PG_PORT")
dbname = os.getenv("PG_DB")

# Créer le moteur SQLAlchemy
engine = create_engine(f"postgresql+psycopg2://{user}:{password}@{host}:{port}/{dbname}")

# Intérroge jobs data dans RDS et l'affiche en Dataframe
df = pd.read_sql("SELECT * FROM jobs_data;", engine)
df.head()

,Job_Title,Company,Location,Description
0,BDD Writer,Qode,usa,BDD Writer (Behavior-Driven Development) with...
1,Environmental Land Planner,"Eocene Environmental Group, Inc.",usa,This description is a summary of our understan...
2,QA Automation Technical Lead,3Pillar Global,romania,Embark on a pivotal journey to the forefront o...
3,HRSS Associate,Safeguard Global Philippines Inc (Philippines),philippines,This description is a summary of our understan...
4,HR Coordinator,TechnologyAdvice,philippines,"Hi, were TechnologyAdvice. At TechnologyAdvice..."


### Traitement des valeurs manquantes, et les doublons

In [3]:
# Informations générales sur les données
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1679 entries, 0 to 1678
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Job_Title    1679 non-null   object
 1   Company      1679 non-null   object
 2   Location     1679 non-null   object
 3   Description  1679 non-null   object
dtypes: object(4)
memory usage: 52.6+ KB


Notre dataset a 4 colonnes et 1679 lignes.

Ne prenant pas en compte la méthode de conversion de données dans mon pipeline, on remarque que toutes les données sont de types: Object.

Prenant en compte la méthode du traitement de données manquantes dans mon pipeline, c'est normal que notre dataset n'a aucune donnée manquante.

In [4]:
# Voir le nombre de valeurs duppliquées
df.duplicated().sum()

9

Notre dataset a 9 données dupliquées.

In [5]:
# Suppression de valeurs dupliquées
df.drop_duplicates(inplace=True)

In [6]:
# Vérification de l'existence des données dupliquées
df.duplicated().sum()

0

Il n'existe plus de données dupliquées

### Pourcentage des jobs les plus demandés

In [7]:
job_counts = df['Job_Title'].value_counts(normalize=True) * 100
job_percent_df = job_counts.reset_index()
job_percent_df.columns = ['Job_Title', 'Percentage']

### Top 5 des localisations par Job

In [8]:
top_countries_by_job = df.groupby(['Job_Title', 'Location']).size().reset_index(name='Count')
top3_per_job = (
    top_countries_by_job
    .sort_values(['Job_Title', 'Count'], ascending=[True, False])
    .groupby('Job_Title')
    .head(5)
)

### Hiérarchie des pays avec le plus d’offres

In [9]:
country_counts = df['Location'].value_counts().reset_index()
country_counts.columns = ['Country', 'Count']

In [11]:
import plotly.express as px

# 1. Top 10 Job Titles les plus demandés
top10_job_percent = job_percent_df.sort_values(by='Percentage', ascending=False).head(10)
fig1 = px.bar(
    top10_job_percent,
    x='Job_Title',
    y='Percentage',
    title='Top 10 des Jobs les plus demandés (%)',
    color='Percentage',
    text='Percentage'
)
fig1.update_traces(texttemplate='%{text:.2f}%', textposition='outside')
fig1.update_layout(yaxis_title='Pourcentage', xaxis_title='Jobs')
fig1.show()

# Trier les pourcentages par ordre croissant
sorted_jobs = job_percent_df.sort_values(by='Percentage').reset_index(drop=True)

# Trouver l'index de la médiane
middle_idx = len(sorted_jobs) // 2

# Extraire 5 avant et 5 après la médiane (ajuster si moins de 10 lignes dans total)
start_idx = max(middle_idx - 5, 0)
end_idx = min(middle_idx + 5, len(sorted_jobs))

median10_job_percent = sorted_jobs.iloc[start_idx:end_idx]

# Trier les pourcentages par ordre croissant
sorted_jobs = job_percent_df.sort_values(by='Percentage').reset_index(drop=True)

# Trouver l'index de la médiane
middle_idx = len(sorted_jobs) // 2

# Extraire 5 avant et 5 après la médiane (ajuster si moins de 10 lignes dans total)
start_idx = max(middle_idx - 5, 0)
end_idx = min(middle_idx + 5, len(sorted_jobs))

median10_job_percent = sorted_jobs.iloc[start_idx:end_idx]

# 2. Top 10 des Job Titles moyennement demandés
fig2 = px.bar(
    median10_job_percent,
    x='Job_Title',
    y='Percentage',
    title='Top 10 des Jobs moyennement demandés (%)',
    color='Percentage',
    text='Percentage'
)
fig2.update_traces(texttemplate='%{text:.2f}%', textposition='outside')
fig2.update_layout(yaxis_title='Pourcentage', xaxis_title='Jobs')
fig2.show()

# 3. Top 10 Job Titles les moins demandés
bottom10_job_percent = job_percent_df.sort_values(by='Percentage', ascending=True).head(10)
fig3 = px.bar(
    bottom10_job_percent,
    x='Job_Title',
    y='Percentage',
    title='Top 10 des Jobs les moins demandés (%)',
    color='Percentage',
    text='Percentage'
)
fig3.update_traces(texttemplate='%{text:.2f}%', textposition='outside')
fig3.update_layout(yaxis_title='Pourcentage', xaxis_title='Jobs')
fig3.show()

# 4. Top 5 des localisation avec le plus d’offres
top5_countries = country_counts.sort_values(by='Count', ascending=False).head(5)
fig4 = px.bar(
    top5_countries,
    x='Country',
    y='Count',
    title='Top 5 des localisations avec le plus d\'offres',
    color='Country',
    text='Count'
)
fig4.update_traces(textposition='outside')
fig4.update_layout(yaxis_title='Nombre d\'offres', xaxis_title='localisation')
fig4.show()
